In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv('insurance_pre.csv')

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [7]:
dataset = pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [9]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [11]:
independent = dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [13]:
dependent = dataset[['charges']]

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Corrected param_grid
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],  # Valid criteria for regression
    'max_features': ['sqrt', 'log2', None],  # Number of features considered for split
    'max_depth': [None, 10, 20, 30],  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required in a leaf node
    'bootstrap': [True, False],  # Whether to bootstrap samples
}

# Perform grid search
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(x_train, y_train)

# Display results
print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)


Fitting 5 folds for each of 3456 candidates, totalling 17280 fits


C:\Anoconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'bootstrap': True, 'criterion': 'absolute_error', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
Best Score: 0.8411108887330089


In [45]:
result = grid.cv_results_

In [47]:
grid_predictions = grid.predict(x_test)

In [49]:
from sklearn.metrics import r2_score
r_score = r2_score(y_test,grid_predictions)

In [51]:
r_score

0.8905870139484385

In [53]:
table = pd.DataFrame.from_dict(result)